In [ ]:
import json
import os
import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv
from openai import OpenAI
from pathlib import Path
from pprint import pprint
from pydantic import BaseModel, Field
from student_feedback_utils import compute_model_cost
from typing import List, Union

load_dotenv(override=True)
client = OpenAI()

In [ ]:
# Set working directory - Not required if using Jupyter outside of VScode
workdir = os.environ["workdir"]
os.chdir(workdir)
os.getcwd()

In [ ]:
# Load in Data
data_dir = Path("./data/")

student_answers_llm_graded = pd.read_csv(data_dir / "student_answers_llm_graded.csv")

In [ ]:
student_answers = student_answers_llm_graded[(student_answers_llm_graded.question_type == "hs_explain") & (student_answers_llm_graded.llm_mark_hitrate == True)].to_dict(orient="records")
student_answer = [i for i in student_answers if i["student_id"] == 5200][0]
student_answer

In [ ]:
print("Question Text:\n", student_answer["question_text"])
print("-"*40)
print("Answer Text:\n", student_answer["answer_text"])
print("-"*40)
print("Mark Scheme Text:\n", student_answer["mark_scheme_text"])
print("-"*40)
print("LLM Graded Answer:\n", student_answer["llm_graded_answer"])
print("-"*40)


In [ ]:
# Opening JSON file
with open('student_feedback_prompt.json', 'r') as openfile:
 
    # Reading from json file
    student_feedback_extract_prompt = json.load(openfile)["prompt"]

In [ ]:
print(student_feedback_extract_prompt)

In [ ]:
class NumberedListItem(BaseModel):
    """Represents an individual item in the numbered list."""
    number: int
    instruction: str

class StudentFeedback(BaseModel):
    """Student Feedback Format."""
    answer_feedback: str
    advice_to_improve_answer: List[NumberedListItem] = Field(default_factory=list)
    revised_answer: str


#student_answer = full_marks_answer.copy()

if np.isnan( student_answer["llm_level_hitrate"] ):
    print("Student Feedback is only given for level hitrate.")

elif student_answer.get("llm_awarded_marks") == student_answer.get("total_marks"):
    chat_completion = "Good job, you achieved full marks."

else:

    chat_completion = client.beta.chat.completions.parse(
        messages=[
            {
                "role": "system",
                "content": "You are an expert teacher with a specialty in helping students how they can improve their answers to achieve the highest marks."
            },
            {
                "role": "user",
                "content": student_feedback_extract_prompt.format_map(student_answer),
            }
        ],
        response_format=StudentFeedback,
        model="gpt-4o-mini",
    )

In [ ]:
student_feedback_response = json.loads(chat_completion.model_dump_json())
processing_cost = compute_model_cost(student_feedback_response)

print("Total Cost:\n", processing_cost["total_cost"])

In [ ]:
student_feedback = chat_completion if isinstance(chat_completion, str) else chat_completion.choices[0].message.content

In [ ]:
pprint(student_feedback)

In [ ]:
print("LLM_Graded_Answer:\n", student_answer["llm_graded_answer"] )
print("-"*40)
print("Feedback on Student Answer:\n", student_feedback)